In [ ]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# import target, pop, and gdp data per state
y_df = pd.read_csv('data/climatewatch-usemissions.csv', usecols=['State', 'Year', 'Population (People)','State GDP (Million US$ (chained 1997/2005))','Transportation (MtCO2e)'])
y_df.head()

In [ ]:
y_df.count()

In [ ]:
# import gas usage, transit ridership, vehicle miles traveled, and vehicle data per state
transit_df = pd.read_csv('data/transportation_usage.csv', encoding = 'utf-16', sep='\t')
transit_df.head()

In [ ]:
### Create a column for each measure in transit_df['Measures'] and assign the appropriate values
# Grab measure names
measures_list = transit_df['Measures'].unique()

# Limit list of measure names to gas usage, transit ridership, vehicle miles traveled, and vehicle data
measures_list = measures_list[0:4]

# Create initial dataframe based on transit ridership per state and per year
to_merge_df = transit_df.loc[transit_df['Measures']==measures_list[0]]
new_column_name = to_merge_df.iloc[0]['Measures']
to_merge_df.columns=['State', 'Measures', 'Year', new_column_name]
to_merge_df = to_merge_df.drop(columns=['Measures'])
transit_measures_df = to_merge_df

# Merge additional measures (gas usage, vehicle miles traveled, and vehicle data)
for column in measures_list[1:4]:
    to_merge_df = transit_df.loc[transit_df['Measures']== column]
    new_column_name = to_merge_df.iloc[0]['Measures']
    to_merge_df.columns=['State', 'Measures', 'Year', new_column_name]
    to_merge_df = to_merge_df.drop(columns=['Measures'])
    transit_measures_df = transit_measures_df.merge(to_merge_df, how = 'left', on = ['State', 'Year'])
transit_measures_df.head()

In [ ]:
# Remove null values (post csv exploration)
transit_measures_df = transit_measures_df.loc[transit_measures_df['Year'] >= 1994]
transit_measures_df = transit_measures_df.loc[transit_measures_df['State'] != 'Puerto Rico']
transit_measures_df.count()

In [ ]:
# merge y_df and transit_measures_df
final_df = y_df.merge(transit_measures_df, on = ['State', 'Year'])
final_df.head()

In [ ]:
final_df.count()

In [ ]:
# Scrape land area (in sq mi) from:
# https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area
wikiurl = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area#cite_note-2010census-2'

# Check response code to ensure ability to download
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

In [ ]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find('table',{'class':"wikitable"})
wiki_area_df = pd.read_html(str(indiatable))

# convert list to dataframe
wiki_area_df = pd.DataFrame(wiki_area_df[0])

# Drop unwanted level
wiki_area_df.columns = wiki_area_df.columns.droplevel(level=0)
wiki_area_df.head()

# Grab desired column (sq mi in index 5)
area_df = wiki_area_df.iloc[:, [0,5]]
area_df.head()

In [ ]:
# manually add a value for sq mi total for US
us_df = wiki_area_df.loc[wiki_area_df['State']=='50 states and District of Columbia']
us_sqmi_df = us_df.iloc[[0], [0, 5]]
us_sqmi_df['State'] = us_sqmi_df['State'].replace({'50 states and District of Columbia': 'United States'})
area_df = area_df.append(us_sqmi_df)
area_df

In [ ]:
final_df = final_df.merge(area_df, how='left', on = ['State'])
final_df